### convertir image bmp en jpg

In [ ]:
import os
from PIL import Image

def convert_bmp_to_jpg(folder):
    for file in os.listdir(folder):
        if file.lower().endswith(".bmp"):
            path = os.path.join(folder, file)
            new_file = os.path.splitext(file)[0] + ".jpg"
            new_path = os.path.join(folder, new_file)

            img = Image.open(path)
            img.convert("RGB").save(new_path, "JPEG")
            os.remove(path)
            print(f"{file} → {new_file}")

folder = "data/clean_image"
convert_bmp_to_jpg(folder) 

### Detecter les couleurs normal dans une images

In [2]:
from PIL import Image

# Chemin vers l'image
def extraire_Couleur_conforme(image_path): 
    # Ouvrir l'image et convertir en RGB
    img = Image.open(image_path).convert("RGB")
    # Récupérer les pixels et supprimer les doublons
    unique_colors = set(img.getdata())
    print(f"Nombre de couleurs uniques dans l'image : {len(unique_colors)}")
    # Afficher chaque couleur unique
    print("Couleurs RGB uniques dans l'image :")
    return unique_colors

image_path = 'data/clean_image/30mm_BonW_1.jpg'  
couleurs_uniques = extraire_Couleur_conforme(image_path)
print(couleurs_uniques)


Nombre de couleurs uniques dans l'image : 241
Couleurs RGB uniques dans l'image :
{(227, 227, 227), (198, 198, 198), (13, 13, 13), (236, 236, 236), (41, 41, 41), (20, 20, 20), (89, 89, 89), (100, 100, 100), (231, 231, 231), (55, 55, 55), (29, 29, 29), (50, 50, 50), (122, 122, 122), (143, 143, 143), (8, 8, 8), (67, 67, 67), (88, 88, 88), (181, 181, 181), (202, 202, 202), (212, 212, 212), (65, 65, 65), (219, 219, 219), (240, 240, 240), (5, 5, 5), (160, 160, 160), (72, 72, 72), (43, 43, 43), (64, 64, 64), (93, 93, 93), (124, 124, 124), (38, 38, 38), (110, 110, 110), (102, 102, 102), (123, 123, 123), (131, 131, 131), (152, 152, 152), (162, 162, 162), (183, 183, 183), (169, 169, 169), (161, 161, 161), (190, 190, 190), (221, 221, 221), (14, 14, 14), (168, 168, 168), (189, 189, 189), (228, 228, 228), (52, 52, 52), (73, 73, 73), (104, 104, 104), (112, 112, 112), (80, 80, 80), (90, 90, 90), (111, 111, 111), (132, 132, 132), (140, 140, 140), (171, 171, 171), (192, 192, 192), (25, 25, 25), (118, 

In [ ]:
import cv2
import numpy as np

def verifier_image_conformite(image_path, unique_colors):
    """
    Vérifie si une image contient une couleur non présente dans le set unique_colors
    à l'intérieur du cercle du décalque.
    """
    # 1. Charger l'image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image introuvable : {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 2. Grayscale et blur pour la détection de cercle
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, 5)

    # 3. Détection du cercle
    circles = cv2.HoughCircles(
        gray_blur,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=500,
        param1=100,
        param2=80,
        minRadius=200,
        maxRadius=1000
    )

    if circles is None:
        print("❌ Aucun cercle détecté.")
        return

    circles = np.uint16(np.around(circles))
    x, y, r = circles[0][0]

    # 4. Masque du cercle
    mask = np.zeros_like(gray)
    cv2.circle(mask, (x, y), r, 255, -1)

    # 5. Extraire les pixels dans le cercle
    pixels_cercle = image_rgb[mask == 255]

    # 6. Vérification des couleurs
    couleurs_intruses = set()
    for pix in pixels_cercle:
        pix_tuple = tuple(pix)
        if pix_tuple not in unique_colors:
            couleurs_intruses.add(pix_tuple)
            if len(couleurs_intruses) >= 1:
                break  # dès qu'on détecte une couleur non conforme

    # 7. Résultat
    if couleurs_intruses:
        print("❌ NON CONFORME : couleur(s) intruse(s) détectée(s) :")
        for c in couleurs_intruses:
            print(f"  RGB{c}")
    else:
        print("✅ CONFORME : aucune couleur en surplus détectée.")


image_path = 'data/clean_image/image_3.jpg'
verifier_image_conformite(image_path, unique_colors)


❌ NON CONFORME : couleur(s) intruse(s) détectée(s) :
  RGB(141, 141, 143)
